In [1]:
import seaborn as sns
from pydataset import data
import pandas as pd
from pandas import DataFrame
import acquire1
# import splitting and imputing functions
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer


In a jupyter notebook, classification_exercises.ipynb, use a python module (pydata or seaborn datasets) containing datasets as a source from the iris data. Create a pandas dataframe, df_iris, from this data.

In [ ]:
iris = data('iris')

In [ ]:
iris.head()

In [ ]:
#print the first 3 rows
iris.head(3)

In [ ]:
#print the number of rows and columns (shape)
iris.shape

In [ ]:
#print the data type of each column
iris.dtypes

In [ ]:
#print the summary statistics for each of the numeric variables. Would you recommend 
#rescaling the data based on these statistics? 
#rescaling means normalize the data within a particular range, I don't think they need to be 
# rescaled because they seem to be in a normalized range.
iris.describe().T

2. Read the Table1_CustDetails table from the Excel_Exercises.xlsx file into a dataframe named df_excel.


In [ ]:
#read the csv file and name it df_excel as instructed
df_excel = pd.read_csv("Telco.csv")

In [ ]:
#show me the dataframe
df_excel

In [ ]:
#assign the first 100 rows to a new dataframe, df_excel_sample
df_excel_sample = df_excel.head(100)
df_excel_sample

In [ ]:
#print the number of rows of your original dataframe
df_excel.shape[0]

In [ ]:
#print the first 5 column names
df_excel.columns[0:5]

In [ ]:
#print the column names that have a data type of object
df_excel.select_dtypes

In [ ]:
#print the summary statistics for each of the numeric variables
df_excel.describe()

In [ ]:
#print the unique values for each of your categorical variables
df_excel.nunique()

3. Read the data from this google sheet into a dataframe, df_google

In [ ]:
df_google = pd.read_csv("googlesheets.csv")

In [ ]:
df_google

In [ ]:
#print the first 3 rows
print(df_google.iloc[:3])

In [ ]:
#print the number of rows and columns
df_google.shape

In [ ]:
#print the column names
df_google.columns

In [ ]:
#print the data type of each column
df_google.dtypes.T

In [ ]:
#print the summary statistics for each of the numeric variables
df_google.describe()

In [ ]:
#print the unique values for each of your categorical variables

df_google.nunique()

In [ ]:
######################################################## PREPARE

In [ ]:
import env

In [ ]:
def new_iris_data():
    '''
    This function reads the iris data from the Codeup db into a df.
    '''
    sql_query = """
                SELECT 
                    species_id,
                    species_name,
                    sepal_length,
                    sepal_width,
                    petal_length,
                    petal_width
                FROM measurements
                JOIN species USING(species_id)
                """
    
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('iris_db'))
    
    return df


def get_iris_data():
    '''
    This function reads in iris data from Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('iris_df.csv'):
        
        # If csv file exists read in data from csv file.
        df = pd.read_csv('iris_df.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame
        df = new_iris_data()
        
        # Cache data
        df.to_csv('iris_df.csv')
        
    return df

In [ ]:
df= acquire1.get_iris_data()

In [ ]:
df

In [ ]:
#Drop the species_id and (this one wasn't there to drop)measurement_id columns.
df = df.drop(columns = ['species_id'])

In [ ]:
df

In [ ]:
#Rename the species_name column to just species.
df.rename(columns={'species_name': 'species'}, inplace=True)


In [ ]:
df

In [ ]:
#Create dummy variables of the species name.
# use pd.get_dummies. Returns a dataframe
df_dummy = pd.get_dummies(df[['species']]) #drop_first=[True] if I want to whittle down columns to only the necessary

df_dummy.head()

In [ ]:
df= pd.concat([df, df_dummy], axis = 1)
df.head()

In [ ]:
#Create a function named prep_iris that accepts the untransformed iris data, and returns 
#the data with the transformations above applied.
def prep_iris(iris_db):
    df= acquire1.get_iris_data()
    df = df.drop(columns = ['species_id'])    
    df.rename(columns={'species_name': 'species'}, inplace=True)    
    df_dummy = pd.get_dummies(df[['species']])
    df= pd.concat([df, df_dummy], axis = 1)  
    return df

In [ ]:
########################### PRACTICE

In [ ]:
#read the csv file and name it df_telco to practice with
df_telco = pd.read_csv("Telco.csv")

In [ ]:
#Clean data

In [ ]:
#determine what columns to drop, no use for customer_id
df_telco = df_telco.drop(columns= "customer_id")

In [ ]:
#Check to make sure customer_id column was dropped
df_telco.columns

In [ ]:
df_telco.shape

In [ ]:
#drop any duplicates
df_telco = df_telco.drop_duplicates()

In [ ]:
#check to see if any duplicates were dropped
df_telco.shape

In [ ]:
#Check for null values to determine which columns to drop or nulls to replace
df_telco.isnull().sum()

In [ ]:
# Find the average monthly charge to replace null values
df_telco.total_charges.mean()

In [ ]:
# Replace null values with average monthly charge
df_telco.fillna(value = 2283.30, inplace = True)

In [ ]:
#Check to see if null values are now filled with the average
df_telco.isnull().sum()

In [ ]:
df_telco.head(20)

In [ ]:
#Get dummy values dependents and churn columns; unnecessary columns
dummy_df = pd.get_dummies(df_telco[['dependents','churn']], dummy_na=False, drop_first=[True,True])
dummy_df.head()

In [ ]:
#attach the dummy columns to the main df
df_telco = pd.concat([df_telco,dummy_df], axis=1)
df_telco.head(1)

In [ ]:
#this one didn't work but not sure why

#drop the columns we just make dummy columns for
dropping = ['dependents','churn']
df_telco.drop(columns= dropping)
df_telco.head()

In [ ]:
#time to train, validate, test, split

In [ ]:
# 20% test, 80% train_validate
# then of the 80% train_validate: 30% validate, 70% train. 

train, test = train_test_split(df_telco, test_size=.2, random_state=123, stratify=df_telco.churn_Yes)
train, validate = train_test_split(train, test_size=.3, random_state=123, stratify=train.churn_Yes)

In [ ]:
#validate the split
print(f'train -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test -> {test.shape}')

In [ ]:
# creating a function that will train, validate and split data when a df in input
def split_data(df_telco):
    train, test = train_test_split(df_telco, test_size=.2, random_state=123, stratify=df_telco.churn_Yes)
    train, validate = train_test_split(train, test_size=.3, random_state=123, stratify=train.churn_Yes)
    return train, validate, test

In [ ]:
#test the function
train, validate, test = split_data(df_telco)

In [ ]:
#validate the split

print(f'train -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test -> {test.shape}')

In [ ]:
################################## EXPLORATORY DATA ANALYSIS

In [ ]:
# ACQUIRE

In [2]:
from env import get_connection

In [4]:
def new_iris_data():
    '''
    This function reads the iris data from the Codeup db into a df.
    '''
    sql_query = """
                SELECT 
                    species_id,
                    species_name,
                    sepal_length,
                    sepal_width,
                    petal_length,
                    petal_width
                FROM measurements
                JOIN species USING(species_id)
                """
    
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('iris_db'))
    
    return df


In [9]:
new_iris_data().head()

,species_id,species_name,sepal_length,sepal_width,petal_length,petal_width
0,1,setosa,5.1,3.5,1.4,0.2
1,1,setosa,4.9,3.0,1.4,0.2
2,1,setosa,4.7,3.2,1.3,0.2
3,1,setosa,4.6,3.1,1.5,0.2
4,1,setosa,5.0,3.6,1.4,0.2


In [11]:
def access_iris_data():
    '''
    This function reads the iris data from SQL
    '''
    sql_query = """
                SELECT *
                FROM measurements
                JOIN species USING(species_id);
                """
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('iris_db'))
    
    return df

In [13]:
iris_df =access_iris_data()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy import stats

def train_validate_test_split(df, target, seed=123):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''
    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=seed, 
                                            stratify=df[target])
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed,
                                       stratify=train_validate[target])
    return train, validate, test


def explore_univariate(train, cat_vars, quant_vars):
    for var in cat_vars:
        explore_univariate_categorical(train, var)
        print('_________________________________________________________________')
    for col in quant_vars:
        p, descriptive_stats = explore_univariate_quant(train, col)
        plt.show(p)
        print(descriptive_stats)

def explore_bivariate(train, target, cat_vars, quant_vars):
    for cat in cat_vars:
        explore_bivariate_categorical(train, target, cat)
    for quant in quant_vars:
        explore_bivariate_quant(train, target, quant)

def explore_multivariate(train, target, cat_vars, quant_vars):
    '''
    '''
    plot_swarm_grid_with_color(train, target, cat_vars, quant_vars)
    plt.show()
    violin = plot_violin_grid_with_color(train, target, cat_vars, quant_vars)
    plt.show()
    pair = sns.pairplot(data=train, vars=quant_vars, hue=target)
    plt.show()
    plot_all_continuous_vars(train, target, quant_vars)
    plt.show()    


### Univariate

def explore_univariate_categorical(train, cat_var):
    '''
    takes in a dataframe and a categorical variable and returns
    a frequency table and barplot of the frequencies. 
    '''
    frequency_table = freq_table(train, cat_var)
    plt.figure(figsize=(2,2))
    sns.barplot(x=cat_var, y='Count', data=frequency_table, color='lightseagreen')
    plt.title(cat_var)
    plt.show()
    print(frequency_table)

def explore_univariate_quant(train, quant_var):
    '''
    takes in a dataframe and a quantitative variable and returns
    descriptive stats table, histogram, and boxplot of the distributions. 
    '''
    descriptive_stats = train[quant_var].describe()
    plt.figure(figsize=(8,2))

    p = plt.subplot(1, 2, 1)
    p = plt.hist(train[quant_var], color='lightseagreen')
    p = plt.title(quant_var)

    # second plot: box plot
    p = plt.subplot(1, 2, 2)
    p = plt.boxplot(train[quant_var])
    p = plt.title(quant_var)
    return p, descriptive_stats

def freq_table(train, cat_var):
    '''
    for a given categorical variable, compute the frequency count and percent split
    and return a dataframe of those values along with the different classes. 
    '''
    class_labels = list(train[cat_var].unique())

    frequency_table = (
        pd.DataFrame({cat_var: class_labels,
                      'Count': train[cat_var].value_counts(normalize=False), 
                      'Percent': round(train[cat_var].value_counts(normalize=True)*100,2)}
                    )
    )
    return frequency_table


#### Bivariate

def explore_bivariate_categorical(train, target, cat_var):
    '''
    takes in categorical variable and binary target variable, 
    returns a crosstab of frequencies
    runs a chi-square test for the proportions
    and creates a barplot, adding a horizontal line of the overall rate of the target. 
    '''
    print(cat_var, "\n_____________________\n")
    ct = pd.crosstab(train[cat_var], train[target], margins=True)
    chi2_summary, observed, expected = run_chi2(train, cat_var, target)
    p = plot_cat_by_target(train, target, cat_var)

    print(chi2_summary)
    print("\nobserved:\n", ct)
    print("\nexpected:\n", expected)
    plt.show(p)
    print("\n_____________________\n")

def explore_bivariate_quant(train, target, quant_var):
    '''
    descriptive stats by each target class. 
    compare means across 2 target groups 
    boxenplot of target x quant
    swarmplot of target x quant
    '''
    print(quant_var, "\n____________________\n")
    descriptive_stats = train.groupby(target)[quant_var].describe()
    average = train[quant_var].mean()
    mann_whitney = compare_means(train, target, quant_var)
    plt.figure(figsize=(4,4))
    boxen = plot_boxen(train, target, quant_var)
    swarm = plot_swarm(train, target, quant_var)
    plt.show()
    print(descriptive_stats, "\n")
    print("\nMann-Whitney Test:\n", mann_whitney)
    print("\n____________________\n")

## Bivariate Categorical

def run_chi2(train, cat_var, target):
    observed = pd.crosstab(train[cat_var], train[target])
    chi2, p, degf, expected = stats.chi2_contingency(observed)
    chi2_summary = pd.DataFrame({'chi2': [chi2], 'p-value': [p], 
                                 'degrees of freedom': [degf]})
    expected = pd.DataFrame(expected)
    return chi2_summary, observed, expected

def plot_cat_by_target(train, target, cat_var):
    p = plt.figure(figsize=(2,2))
    p = sns.barplot(cat_var, target, data=train, alpha=.8, color='lightseagreen')
    overall_rate = train[target].mean()
    p = plt.axhline(overall_rate, ls='--', color='gray')
    return p


## Bivariate Quant

def plot_swarm(train, target, quant_var):
    average = train[quant_var].mean()
    p = sns.swarmplot(data=train, x=target, y=quant_var, color='lightgray')
    p = plt.title(quant_var)
    p = plt.axhline(average, ls='--', color='black')
    return p

def plot_boxen(train, target, quant_var):
    average = train[quant_var].mean()
    p = sns.boxenplot(data=train, x=target, y=quant_var, color='lightseagreen')
    p = plt.title(quant_var)
    p = plt.axhline(average, ls='--', color='black')
    return p

# alt_hyp = ‘two-sided’, ‘less’, ‘greater’

def compare_means(train, target, quant_var, alt_hyp='two-sided'):
    x = train[train[target]==0][quant_var]
    y = train[train[target]==1][quant_var]
    return stats.mannwhitneyu(x, y, use_continuity=True, alternative=alt_hyp)


### Multivariate

def plot_all_continuous_vars(train, target, quant_vars):
    '''
    Melt the dataset to "long-form" representation
    boxenplot of measurement x value with color representing the target variable. 
    '''
    my_vars = [item for sublist in [quant_vars, [target]] for item in sublist]
    sns.set(style="whitegrid", palette="muted")
    melt = train[my_vars].melt(id_vars=target, var_name="measurement")
    plt.figure(figsize=(8,6))
    p = sns.boxenplot(x="measurement", y="value", hue=target, data=melt)
    p.set(yscale="log", xlabel='')    
    plt.show()

def plot_violin_grid_with_color(train, target, cat_vars, quant_vars):
    cols = len(cat_vars)
    for quant in quant_vars:
        _, ax = plt.subplots(nrows=1, ncols=cols, figsize=(16, 4), sharey=True)
        for i, cat in enumerate(cat_vars):
            sns.violinplot(x=cat, y=quant, data=train, split=True, 
                           ax=ax[i], hue=target, palette="Set2")
            ax[i].set_xlabel('')
            ax[i].set_ylabel(quant)
            ax[i].set_title(cat)
        plt.show()

def plot_swarm_grid_with_color(train, target, cat_vars, quant_vars):
    cols = len(cat_vars)
    for quant in quant_vars:
        _, ax = plt.subplots(nrows=1, ncols=cols, figsize=(16, 4), sharey=True)
        for i, cat in enumerate(cat_vars):
            sns.swarmplot(x=cat, y=quant, data=train, ax=ax[i], hue=target, palette="Set2")
            ax[i].set_xlabel('')
            ax[i].set_ylabel(quant)
            ax[i].set_title(cat)
        plt.show()



In [ ]:
#PREPARE

In [14]:
iris_df.head()

,species_id,measurement_id,sepal_length,sepal_width,petal_length,petal_width,species_name
0,1,1,5.1,3.5,1.4,0.2,setosa
1,1,2,4.9,3.0,1.4,0.2,setosa
2,1,3,4.7,3.2,1.3,0.2,setosa
3,1,4,4.6,3.1,1.5,0.2,setosa
4,1,5,5.0,3.6,1.4,0.2,setosa


In [17]:
#drop unnecessary column
iris_df = iris_df.drop(columns = ['species_id'])

In [18]:
iris_df.head()

,measurement_id,sepal_length,sepal_width,petal_length,petal_width,species_name
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa


In [21]:
# sample 
# rename df.rename(columns={'species_name': 'species'}, inplace=True)
iris_df.rename(columns={'species_name':'species'}, inplace=True)

In [22]:
# create dummy columns
df_dummy = pd.get_dummies(iris_df[['species']]) #drop_first=[True] if I want to whittle down columns to only the necessary

df_dummy.head()

,species_setosa,species_versicolor,species_virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [24]:
#add this df to the original
iris_df= pd.concat([iris_df, df_dummy], axis = 1)
iris_df.head()

,measurement_id,sepal_length,sepal_width,petal_length,petal_width,species,species_setosa,species_versicolor,species_virginica
0,1,5.1,3.5,1.4,0.2,setosa,1,0,0
1,2,4.9,3.0,1.4,0.2,setosa,1,0,0
2,3,4.7,3.2,1.3,0.2,setosa,1,0,0
3,4,4.6,3.1,1.5,0.2,setosa,1,0,0
4,5,5.0,3.6,1.4,0.2,setosa,1,0,0


In [25]:
#drop original species column
iris_df = iris_df.drop(columns=['species'])

In [26]:
iris_df

,measurement_id,sepal_length,sepal_width,petal_length,petal_width,species_setosa,species_versicolor,species_virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0
5,6,5.4,3.9,1.7,0.4,1,0,0
6,7,4.6,3.4,1.4,0.3,1,0,0
7,8,5.0,3.4,1.5,0.2,1,0,0
8,9,4.4,2.9,1.4,0.2,1,0,0
9,10,4.9,3.1,1.5,0.1,1,0,0
